In [1]:
number_of_shots = 5

In [2]:
experiment_name = "2024.05.08-02_Mixtral_7B_instruct_API_5-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
# top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11---
1 6099 0  ---SDG 0--- 

Explanation:
The text is about a model's photo shoot for the Sports Illustrated Swimsuit issue in the Maldives. It does not contain any information related to the Sustainable Development Goals. Therefore, the appropriate classification is "---SDG 0---".
2 6471 0  ---SDG 16---

The text discusses the suspension of a life sentence awarded to a convict in a terror-related case and the court's observation on the lack of prima facie evidence to prove the convict's involvement in collecting funds for terror activities in India. The focus of the text is on the legal process and the administration of justice, which aligns with SDG 16: Peace, Justice and Strong Institutions.
3 1786 0  ---SDG 9--- (Industry, Innovation and Infrastructure) as the text is about the discovery and naming of minor planets, which involves innovation and scientific advancement.
4 2143 0  ---SDG 7---
5 3377 0  ---SDG 0---

The text provided is about entertainment news relat

59 3751 0  ---SDG 3---
60 6573 1  ---SDG 1---
61 357 1  ---SDG 1---
62 175 1  ---SDG 1---
63 1446 1  ---SDG 1---
64 6409 1  ---SDG 1---
65 4176 1  ---SDG 1---

The text discusses issues related to poverty and income inequality, which aligns with Sustainable Development Goal (SDG) 1: No Poverty. The specific mention of income quintiles, subsidies, and political purposes further supports this classification. The text does not mention any other SDGs.
66 6655 1  ---SDG 1---
 (This text is discussing poverty and methods for identifying poor individuals, which aligns with the focus of SDG 1: No Poverty.)
67 3587 1  ---SDG 1---
68 1658 1  ---SDG 1---
69 6667 1  ---SDG 1---
70 4588 1  ---SDG 2---
71 4057 1  ---SDG 1---
72 5400 1  ---SDG 1---
73 2631 1  ---SDG 1---

The text discusses the concept of chronic poverty and how it can be measured using severe poverty as a proxy. It also highlights the importance of panel data in analyzing poverty over time. This topic aligns with Sustainable Develop

106 6684 1  ---SDG 1---
107 173 1  ---SDG 1---
108 6136 1  ---SDG 1---

The text discusses poverty and monetary and multidimensional poverty indicators, which aligns with SDG 1: No Poverty. The specific focus on poverty rates, vulnerability to falling into poverty, and disparities between urban and rural households also supports this classification.
109 2652 1  ---SDG 1---
 (The text is about poverty reduction and social assistance benefits, which aligns with SDG 1: No Poverty)
110 1328 1  ---SDG 1---
111 3124 1  ---SDG 1--- (This text is about reducing poverty and improving socio-economic conditions for children, which aligns with SDG 1: No Poverty)
112 3905 1  ---SDG 11---
113 3611 1  ---SDG 1---
114 1293 1  ---SDG 1---

The text emphasizes the importance of information for guiding integrated policies and mentions the measurement of poverty, which aligns with SDG 1: No Poverty.
115 1546 1  ---SDG 1---

The text emphasizes the importance of addressing poverty and the difficulties gove

149 2827 2  ---SDG 2--- (The text discusses support for the agricultural sector, which aligns with SDG 2's goal to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture.)
150 223 2  ---SDG 2---

The text discusses the projected trends in agricultural commodity prices, particularly for cereals, oilseeds, sugar, poultry meat, dairy products, beef/veal, and pigmeat. It highlights the impact of various factors such as droughts, bioenergy policies, and price increases for agricultural inputs on these prices. This information is directly related to SDG 2, which aims to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture.
151 1319 2  ---SDG 2--- (Sustainable Development Goal 2 focuses on ending hunger, achieving food security and improved nutrition, and promoting sustainable agriculture.) The text discusses the increase in agricultural production, particularly in middle-income countries, and the growth in cereal

176 4757 2  ---SDG 13---
177 5518 2  ---SDG 2--- (This text discusses food security and children's well-being, which aligns with SDG 2's focus on ending hunger, achieving food security, and improving nutrition.)
178 2561 2  ---SDG 2--- | Food security, improved productivity in agriculture, and addressing international market imbalances are key points in this text, aligning it with SDG 2.
179 331 2  ---SDG 2--- (Sustainable Development Goal 2 aims to end hunger, achieve food security and improved nutrition and promote sustainable agriculture.)
180 4907 3  ---SDG 3---
181 571 3  ---SDG 3---
182 4948 3  ---SDG 3---
183 5101 3  ---SDG 3---
184 2852 3  ---SDG 3---
185 6575 3  ---SDG 3--- (This text is about healthcare expenditure, which aligns with SDG 3: Good Health and Well-being.)
186 1303 3  ---SDG 3--- (Goal 3 is "Good Health and Well-being")
187 6582 3  ---SDG 3---
188 5625 3  ---SDG 3--- as the text discusses healthcare performance metrics and the need for better adjustment for patie

...

774 1134 12  ---SDG 9---

The text discusses the accreditation of organizations for issuing ISO certificates, specifically ISO 14001, and mentions the certification of three companies by the National Chamber of Commerce and Industry. This relates to the promotion of infrastructure and industrial development, which is covered under SDG 9.
775 1049 12  ---SDG 12---
776 430 12  ---SDG 12---
(This text is about recycling and ecodesign, which aligns with SDG 12's focus on responsible consumption and production.)
777 2354 12  ---SDG 12--- (This text is about waste management and recycling, which aligns with SDG 12's focus on responsible consumption and production.)
778 141 12  ---SDG 12---
779 126 12  ---SDG 12---
780 3805 13  ---SDG 13---
781 2862 13  ---SDG 13---
782 2180 13  ---SDG 13---
783 2676 13  ---SDG 13---
784 5685 13  ---SDG 13---

The text discusses the role of sea ice in reflecting light and limiting warming, and the effects of its melting on temperature increase. This is relate

842 2475 14  ---SDG 14---

The text emphasizes the importance of sustainable management and restoration of marine ecosystems and fish stocks, which are the focus of targets 14.2 and 14.4. The text also highlights the role of science and technology (target 14.a) and implementation of international law (target 14.c) in affecting most of the other targets.
843 229 14  ---SDG 14---
844 925 14  ---SDG 14---
845 4721 14  ---SDG 14--- (This text is about improving the seafood trading environment and expanding seafood exports, which relates to the conservation and sustainable use of the oceans, seas, and marine resources, which is covered by SDG 14.)
846 6196 14  ---SDG 14---
847 432 14  ---SDG 14---

(The text is about a project for the development of a sustainable coastal fishery that does not threaten the marine environment, which aligns with SDG 14: Life Below Water.)
848 4876 14  ---SDG 14--- (This text discusses the challenges and potential changes needed in fishery management, particula

919 3281 15  ---SDG 15---
920 1767 15  ---SDG 11--- \\
This text pertains to SDG 11 (Sustainable Cities and Communities) because it discusses land-use planning and zoning principles to promote development that simultaneously protects and conserves the environment. The text also mentions the need for further efforts to ensure that regions with the highest development potential are covered by ELUPs (Environmental Land-Use Plans).
921 1957 15  ---SDG 13---
922 980 15  ---SDG 15---
923 1299 15  ---SDG 15---
924 5980 15  ---SDG 15---
925 2985 15  ---SDG 15--- (Life on Land) as the text refers to the study and classification of various plant and animal species, including endemic and subendemic species, which falls under SDG 15.
926 1515 15  ---SDG 15---
927 6479 15  ---SDG 15---
928 2296 15  ---SDG 15---
929 3610 15  ---SDG 15---
930 152 15  ---SDG 15---
931 658 15  ---SDG 15---

(The text discusses the value of biodiversity and ecosystem services, which relates to SDG 15: Life on Land, spec

973 3540 16  ---SDG 16---
974 283 16  ---SDG 16---
975 4891 16  ---SDG 8---

The text discusses the impact of deregulation on racial employment in the airline industry. The focus is on the economic theory of how deregulation can reduce employers' ability to discriminate when hiring and the findings of a specific study on the airline industry. This topic falls under SDG 8, which is to "promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all." The text does not mention any other SDGs, and the topic is not relevant to SDG 0, which is about "no poverty."
976 3236 16  ---SDG 16---
977 6715 16  ---SDG 17---

The reason for this classification is that the text discusses the impact of geopolitics, specifically the balance of power between countries, on the response to the covid-19 pandemic. This topic relates to SDG 17, which aims to strengthen global partnerships and cooperation to achieve sustainable development goals. The text hig

1017 4563 16  ---SDG 16---
1018 6710 16  ---SDG 16---
1019 1947 16  ---SDG 16---


In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9---\n\nThe text is about the analysis of developing activities from 77 providers, which is assumed to support STI. The text mentions core funding to a multilateral agency, which may indirectly support STI-related activities. However, it is difficult to determine the share of core funding dedicated to these activities. Therefore, the text can be classified under SDG 9, which focuses on building resilient infrastructure, promoting sustainable industrialization, and fostering innovation.')
('2', 6471, 0, " ---SDG 16---\n\nThe text discusses the suspension of a life sentence awarded to a convict in a terror-related case and the court's observation on the lack of prima facie evidence to prove the convict's involvement in collecting funds for terror activities in India. The focus of the text is on the legal process and the administration of justice, which aligns with SDG 16: Peace, Justice and Strong Institutions.")
('782', 2180, 

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed.head(3)

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9---\n\nThe text is about the analysis...
2,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,30,30,1,30,30,1,1,1,30,30,30,30,30
1,72,72,71,72,72,71,71,71,72,72,72,72,72
10,32,32,31,32,32,31,31,31,32,32,32,32,32
11,83,83,77,83,83,77,77,77,83,83,83,83,83
12,55,55,55,55,55,55,55,55,55,55,55,55,55
13,59,59,58,59,59,58,58,58,59,59,59,59,59
14,53,53,53,53,53,53,53,53,53,53,53,53,53
15,57,57,55,57,57,55,55,55,57,57,57,57,57
16,51,51,47,51,51,47,47,47,51,51,51,51,51


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.97      0.48      0.64        60
           1       0.71      0.85      0.77        60
           2       0.84      0.78      0.81        60
           3       0.82      0.93      0.87        60
           4       0.83      0.90      0.86        60
           5       0.85      0.83      0.84        60
           6       0.98      0.87      0.92        60
           7       0.86      0.83      0.85        60
           8       0.54      0.73      0.62        60
           9       0.62      0.78      0.69        60
          10       0.88      0.47      0.61        60
          11       0.70      0.97      0.81        60
          12       0.89      0.82      0.85        60
          13       0.83      0.82      0.82        60
          14       0.96      0.85      0.90        60
          15       0.93      0.88      0.91        60
          16       0.92      0.78      0.85        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- \n\nExplanation:\nThe text is abo...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the suspen...,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.71      0.48      0.57        60
           1       0.71      0.85      0.77        60
           2       0.84      0.78      0.81        60
           3       0.82      0.93      0.87        60
           4       0.83      0.90      0.86        60
           5       0.85      0.83      0.84        60
           6       0.98      0.87      0.92        60
           7       0.86      0.83      0.85        60
           8       0.54      0.73      0.62        60
           9       0.62      0.78      0.69        60
          10       0.88      0.47      0.61        60
          11       0.70      0.97      0.81        60
          12       0.89      0.82      0.85        60
          13       0.83      0.82      0.82        60
          14       0.96      0.85      0.90        60
          15       0.93      0.88      0.91        60
          16       0.92      0.78      0.85        60

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,70.731707,48.333333,57.425743,60.00000
1,70.833333,85.000000,77.272727,60.00000
2,83.928571,78.333333,81.034483,60.00000
3,82.352941,93.333333,87.500000,60.00000
4,83.076923,90.000000,86.400000,60.00000
5,84.745763,83.333333,84.033613,60.00000
6,98.113208,86.666667,92.035398,60.00000
7,86.206897,83.333333,84.745763,60.00000
8,53.658537,73.333333,61.971831,60.00000
9,61.842105,78.333333,69.117647,60.00000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

81.55158785836868 79.90196078431373 79.90196078431373 79.80261446255324


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

81.6 79.9 79.9 79.8
